# Install stuff on Colab and imports

In [ ]:
#!nvidia-smi

In [ ]:
# In case you want to run it on Google Colab, don't forget to install biome.text first

# !pip install -U pip
# !pip install -U git+https://github.com/recognai/biome-text.git@project/profner
# exit(0)

In [ ]:
# !pip install wandb

In [ ]:
# !wandb login

In [2]:
import itertools
import os
from pathlib import Path

from biome.text import Dataset, Pipeline, TrainerConfiguration
from biome.text.hpo import TuneExperiment
from ray import tune
from ray.tune.suggest.hyperopt import HyperOptSearch

# Create training and validation data sets

In [ ]:
train_ds = Dataset.from_json("../preprocessing_inference/train_v1.json")
valid_ds = Dataset.from_json("../preprocessing_inference/valid_v1.json")

In [ ]:
train_ds.rename_column_("tags_bio", "tags")
valid_ds.rename_column_("tags_bio", "tags")
train_ds.rename_column_("classification_label", "labels")
valid_ds.rename_column_("classification_label", "labels")

In [ ]:
train_ds.head()

# Define pipeline/trainer config and HPO search spaces 

In [7]:
transformers_model: str = "dccuchile/bert-base-spanish-wwm-cased"
#transformers_model: str = "prajjwal1/bert-tiny"

profnert = {
    "name": "profnert",
    "features": {
        "transformers": {
            "model_name": transformers_model,
            "trainable": True,
        }
    },
    "head": {
        "type": "ProfNerT",
        "classification_labels": train_ds.unique("labels"),
        "classification_pooler": {
            "type": "bert_pooler",
            "pretrained_model": transformers_model,
            "requires_grad": True,
            "dropout": 0.1,
        },
        "ner_tags": list(set(itertools.chain.from_iterable(train_ds["tags"]))),
        "ner_tags_encoding": "BIO",
        "transformers_model": transformers_model,
        "dropout": 0.1,
    },
}

In [ ]:
trainer_config = dict(
    optimizer={
        "type": "adamw",
        "lr": tune.loguniform(5e-6, 1e-4),
        "weight_decay": tune.loguniform(1e-3, 1e-1),
    },
    linear_decay=True,
    warmup_steps=tune.randint(0, 200),
    batch_size=tune.choice([8, 16]),
    num_epochs=tune.choice([3, 4, 5]),
    validation_metric="+valid_ner/f1-measure-overall",
    num_serialized_models_to_keep=0,
)

# Launch HPO experiment

In [ ]:
search_alg = HyperOptSearch(metric="validation_valid_ner/f1-measure-overall", mode="max")

# does not support integers
# from ray.tune.suggest.bayesopt import BayesOptSearch
# search_alg = BayesOptSearch(metric="validation_ner/f1-measure-overall", mode="max")

# ray2.0.0 does not use the gpu ...
# from ray.tune.suggest.hebo import HEBOSearch
# search_alg = HEBOSearch(metric="validation_ner/f1-measure-overall", mode="max")

In [ ]:
hpo_experiment = TuneExperiment(
    pipeline_config=profnert,
    trainer_config=trainer_config,
    train_dataset=train_ds,
    valid_dataset=valid_ds,
    name="profner_transformers",
    num_samples=50,
    local_dir="tune_runs",
    resources_per_trial={"cpu": 2, "gpu": 1},
)

In [ ]:
analysis = tune.run(
    hpo_experiment,
    config=hpo_experiment.config,
    scheduler=tune.schedulers.ASHAScheduler(),
    search_alg=search_alg,
    metric="validation_valid_ner/f1-measure-overall",
    mode="max",
#     progress_reporter=tune.CLIReporter(
#         metric_columns=[
#             "best_validation_valid_ner/f1-measure-overall",
#             "best_validation_valid_classification/accuracy",
#         ],
#         parameter_columns=["trainer.optimizer.lr"],
#     ),
    progress_reporter=tune.JupyterNotebookReporter(overwrite=True)
    verbose=1,
)

# Train the best model on train+validation data set

See "profner_transformers" project in wandb tagged as "HyperOpt" (https://wandb.ai/dcfidalgo/profner_transformers), best run: *zany-deluge-56*; 0.8338, 0.967

In [ ]:
train_full_ds = Dataset.from_datasets([train_ds, valid_ds])

In [ ]:
transformers_model: str = "dccuchile/bert-base-spanish-wwm-cased"
#transformers_model: str = "prajjwal1/bert-tiny"

profnert = {
    "name": "profnert",
    "features": {
        "transformers": {
            "model_name": transformers_model,
            "trainable": True,
        }
    },
    "head": {
        "type": "ProfNerT",
        "classification_labels": train_ds.unique("labels"),
        "classification_pooler": {
            "type": "bert_pooler",
            "pretrained_model": transformers_model,
            "requires_grad": True,
            "dropout": 0.1,
        },
        "ner_tags": list(set(itertools.chain.from_iterable(train_ds["tags"]))),
        "ner_tags_encoding": "BIO",
        "transformers_model": transformers_model,
        "dropout": 0.1,
    },
}

In [ ]:
trainer_config = TrainerConfiguration(
    optimizer={
        "type": "adamw",
        "lr": 3.0343630440731596e-05,
        "weight_decay": 0.0017923056109509059,
    },
    warmup_steps=49,
    linear_decay=True,
    batch_size=8,
    num_epochs=4,
    validation_metric="+valid_ner/f1-measure-overall",
    random_seed=4,  # optimized manually
)

In [ ]:
pipeline = Pipeline.from_config(profnert)

In [ ]:
pipeline.train(
    output="final_transformer_model",
    training=train_full_ds,
    trainer=trainer_config,
)